In [1]:
from collections import defaultdict

import optuna.importance
import pandas as pd
# autoimport
%load_ext autoreload
%autoreload 1

In [2]:
from src.gnn_models import *
%aimport src.gnn_models

In [4]:
PROCESSED_DATA_DIR = r"E:\gnn_data\processed_step_data_global_features"
# PROCESSED_DATA_DIR = r"E:\gnn_data\pyg_data_v2"
dataset = FastSTEPDataset(PROCESSED_DATA_DIR, start_index=0)
get_dataset_stats(dataset)

Fast dataset loaded:
- Total samples: 63043
- Processed successfully: 63043
- Failed processing: 54
Label counts in dataset:
Label 1: 40434 instances
Label 0: 22609 instances
Label 1: 64.14% of total instances
Label 0: 35.86% of total instances
Class weights for loss function: [1.5591581342434584, 2.7884028484231944]


In [5]:
dataset[0]

Data(x=[405, 7], edge_index=[2, 948], y=[1], global_features=[1, 14])

In [7]:
model = GINCombined(
    input_features=dataset[0].x.shape[1],
    global_feature_dim=dataset[0].global_features.shape[1],
    embedding_dim=16,
    hidden_sizes=[256, 256],
    conv_dropout_rate=0.1,
    classifier_dropout_rate=0.1,
    use_layer_norm=True,
    pool_hidden_size=128
)

Created Dynamic GIN model:
- Input features: 7
- Number of hidden layers: 2
- Hidden layer sizes: [256, 256]
- Output classes: 2
- Convolution dropout rate: 0.1
- Classifier dropout rate: 0.1
- Layer normalization: True


In [5]:
all_history = {}

In [8]:
for i in range(1):
    try:
        torch.cuda.empty_cache()
        model_save_path = "gin_model_combined.pth"
        history_save_path = "gin_model_combined_training_history.pkl"
        if Path(model_save_path).exists():
            print(f"Loading model from {model_save_path}")
            model.load_state_dict(torch.load(model_save_path))
        else:
            print(f"Model file {model_save_path} does not exist. Initializing a new model.")
        # save training history

        with open(history_save_path, "rb") as f:
            all_history = joblib.load(f)
        trained_model, history = simple_train_model_v3(
            dataset,
            gnn_model=model,
            num_epochs=50,
            batch_size=12,
            learning_rate=0.0001,
            start_index=0,
            num_graphs_to_use=63000,
        )
        for key in history.keys():
            if not key in all_history:
                all_history[key] = []
            all_history[key].extend(history[key])
        all_history["epoch"] = list(range(1, len(all_history["epoch"]) + 1))
        with open(history_save_path, "wb") as f:
            joblib.dump(all_history, f)
        torch.save(model.state_dict(), model_save_path)
    except Exception as e:
        print(f"An error occurred during training epoch {i}: {e}")
        continue

Loading model from gin_model_combined.pth
Label 1: 40414 instances
Label 0: 22586 instances
Label 1: 64.15% of total instances
Label 0: 35.85% of total instances
Class weights: tensor([2.7893, 1.5589], device='cuda:0')
Splitting dataset into train and validation sets
Train samples: 50400
Validation samples: 12600

Starting training for 50 epochs...


Epoch 1/50 [Train]:   0%|          | 0/4200 [00:00<?, ?it/s]

x shape: torch.Size([65527, 7])
edge_index shape: torch.Size([2, 146120])
batch shape: torch.Size([65527])
global_features shape: torch.Size([12, 14])


Epoch 1/50 [Train]:   0%|          | 3/4200 [00:00<11:48,  5.93it/s, loss=0.1460, acc=86.11%, f1=0.8621]

x shape: torch.Size([63817, 7])
edge_index shape: torch.Size([2, 145736])
batch shape: torch.Size([63817])
global_features shape: torch.Size([12, 14])
x shape: torch.Size([95164, 7])
edge_index shape: torch.Size([2, 227586])
batch shape: torch.Size([95164])
global_features shape: torch.Size([12, 14])


...

Epoch 1/50 [Train]:   0%|          | 12/4200 [00:01<07:40,  9.09it/s, loss=0.1793, acc=87.18%, f1=0.8726]

x shape: torch.Size([33555, 7])
edge_index shape: torch.Size([2, 75390])
batch shape: torch.Size([33555])
global_features shape: torch.Size([12, 14])
x shape: torch.Size([34735, 7])
edge_index shape: torch.Size([2, 80012])
batch shape: torch.Size([34735])
global_features shape: torch.Size([12, 14])
x shape: torch.Size([15593, 7])
edge_index shape: torch.Size([2, 36416])
batch shape: torch.Size([15593])
global_features shape: torch.Size([12, 14])


Epoch 1/50 [Train]:   0%|          | 14/4200 [00:01<09:09,  7.62it/s, loss=0.2030, acc=86.90%, f1=0.8698]


x shape: torch.Size([50032, 7])
edge_index shape: torch.Size([2, 112184])
batch shape: torch.Size([50032])
global_features shape: torch.Size([12, 14])


KeyboardInterrupt: 

In [8]:
all_history

{'train_loss': [0.6210318155693156,
  0.5884338162484624,
  0.5807581246058856,
  0.5784128614000621,
  0.5751571377641743,
  0.5726831243905638,
  0.5708053694754129,
  0.5699995922900382,
  0.567597364683946,
  0.5667620221028725,
  0.5674510450155608,
  0.5656477987482434,
  0.5645915722554283,
  0.5617489199074251,
  0.5628458397702447,
  0.5620067406915837,
  0.559581496432601,
  0.5604556760866017,
  0.5597009777730064,
  0.5600156294989089,
  0.5582568416744471,
  0.5587369639506298,
  0.557835575489416,
  0.5562462923622558,
  0.5561987818192159,
  0.5555666007083796,
  0.5553133543775904,
  0.5549816019389601,
  0.5563397216601741,
  0.5559190037278902,
  0.5546640211006715,
  0.5552630360176166,
  0.5537093390675173,
  0.5547770561242388,
  0.555516376777419,
  0.5539051185841007,
  0.5538902622958024,
  0.5519740365392395,
  0.5521404327993237,
  0.5517584696465305,
  0.5518727227344754,
  0.5511733574863701,
  0.5518009466527118,
  0.6181215317653758,
  0.5722680032608055,


In [9]:
torch.save(model.state_dict(), model_save_path)

In [10]:
torch.cuda.empty_cache()
model_save_path = "dynamic_gin_embedding_model.pth"
torch.save(model.state_dict(), model_save_path)
trained_model, history = simple_train_model_v2(
        dataset_trim,
        gnn_model=model,
        num_epochs=5,
        batch_size=4,
        learning_rate=0.002,
        start_index=0,
        num_graphs_to_use=60000,
    )

Label 1: 38182 instances
Label 0: 21818 instances
Label 1: 63.64% of total instances
Label 0: 36.36% of total instances
Class weights: tensor([2.7500, 1.5714], device='cuda:0')
Splitting dataset into train and validation sets
Train samples: 48000
Validation samples: 12000

Starting training for 5 epochs...


Epoch 1/5 [Val]: 100%|██████████| 3000/3000 [00:50<00:00, 59.09it/s, loss=0.6085, acc=81.62%] 


Epoch 1/5 - Train Loss: 0.4351, Acc: 81.23% | Val Loss: 0.4540, Acc: 81.62% (Best Val: 81.62%)


Epoch 2/5 [Val]: 100%|██████████| 3000/3000 [02:21<00:00, 21.18it/s, loss=0.5406, acc=82.01%] 


Epoch 2/5 - Train Loss: 0.4342, Acc: 81.33% | Val Loss: 0.4338, Acc: 82.01% (Best Val: 82.01%)


...

Epoch 3/5 - Train Loss: 0.4330, Acc: 81.44% | Val Loss: 0.4391, Acc: 81.19% (Best Val: 82.01%)


Epoch 4/5 [Val]: 100%|██████████| 3000/3000 [02:22<00:00, 21.02it/s, loss=0.5135, acc=82.33%] 


Epoch 4/5 - Train Loss: 0.4297, Acc: 81.44% | Val Loss: 0.4412, Acc: 82.33% (Best Val: 82.33%)


Epoch 5/5 [Val]: 100%|██████████| 3000/3000 [00:49<00:00, 60.11it/s, loss=0.3723, acc=80.85%] 


Epoch 5/5 - Train Loss: 0.4255, Acc: 81.76% | Val Loss: 0.4389, Acc: 80.85% (Best Val: 82.33%)
Training completed!
Training time: 1849.739828900012
Best validation accuracy: 82.33%


In [26]:
import joblib
import optuna
optuna_result = r"optuna_results_20250820_213139.pkl"
with open(optuna_result, "rb") as f:
    optuna_results = joblib.load(f)

In [18]:
[item.values for item in optuna_results["fold_0"]["study"].trials]

[[0.5],
 [0.5],
 [0.9350761771202087],
 [0.5],
 [0.9295566082000732],
 [0.9362394213676453],
 [0.9269653558731079],
 [0.9116346836090088],
 [0.5],
 [0.93639075756073],
 [0.9339721202850342],
 [0.93220055103302],
 [0.9363347887992859],
 [0.9348156452178955],
 [0.9363428354263306],
 [0.9329842925071716],
 [0.9324948787689209],
 [0.9326227307319641],
 [0.9329482913017273],
 [0.934323787689209],
 [0.9346429109573364],
 [0.9260116815567017],
 [0.9329156875610352],
 [0.9320093393325806],
 [0.9302478432655334],
 [0.9287922382354736],
 [0.9238104820251465],
 [0.9318323731422424],
 [0.9344959259033203],
 [0.9293798208236694],
 [0.9314045310020447],
 [0.9294335842132568],
 [0.9345998764038086],
 [0.9342455863952637],
 [0.9339947700500488],
 [0.9317659139633179],
 [0.9368284940719604],
 [0.933342456817627],
 [0.9371731281280518],
 [0.9343671798706055],
 [0.929416298866272],
 [0.9372449517250061],
 [0.9312037229537964],
 [0.9377567172050476],
 [0.9371228218078613],
 [0.937757134437561],
 [0.936129

In [20]:
optuna_results

{'fold_0': {'best_params': {'num_layers': 1,
   'hidden_size': 256,
   'conv_dropout_rate': 0.3,
   'classifier_dropout_rate': 0.1,
   'use_layer_norm': False,
   'pool_hidden_size': 256,
   'num_epochs': 200,
   'batch_size': 32,
   'learning_rate': 0.00016847952401702993,
   'optimizer_scheduler': 'ReduceLROnPlateau'},
  'best_score': 0.937757134437561,
  'study': <optuna.study.study.Study at 0x2364c6d1c10>}}

In [23]:
pickled_study = optuna_results["fold_0"]["study"]

In [35]:
pickled_study.study_name

'no-name-f04c1bde-e879-4d38-9aee-bb8a7454aaee'

In [39]:
print(f"Loaded study with {len(pickled_study.trials)} trials")
print(f"Best value: {pickled_study.best_value}")

# Create a new study with SQLite storage
sqlite_study = optuna.create_study(
    study_name="gine_study_fold_00",
    storage="sqlite:///optuna_gine_20252008_215100.db",
    direction="maximize",  # or "minimize" depending on your original study
    load_if_exists=True
)

# Copy all trials from pickled study to SQLite study
for trial in pickled_study.trials:
    # Add each trial to the new study
    sqlite_study.add_trial(trial)

print(f"✅ Converted {len(sqlite_study.trials)} trials to SQLite")
print(f"Best value in SQLite study: {sqlite_study.best_value}")

[I 2025-08-20 22:01:38,586] A new study created in RDB with name: gine_study_fold_00


Loaded study with 50 trials
Best value: 0.937757134437561
✅ Converted 50 trials to SQLite
Best value in SQLite study: 0.937757134437561


In [1]:
import optuna
# Name of the study you originally used
study_name = "optuna_gine_fold_0"

# Path to your SQLite database
storage = "sqlite:///gnn_optuna_database/optuna_gine_fold_0.db"

# Load the study
study = optuna.load_study(study_name=study_name, storage=storage)

# Now you can access the results
print(study.trials[-1]._trial_id)


111


In [3]:
study.best_trial

17

In [41]:
pickled_study.best_params

{'num_layers': 1,
 'hidden_size': 256,
 'conv_dropout_rate': 0.3,
 'classifier_dropout_rate': 0.1,
 'use_layer_norm': False,
 'pool_hidden_size': 256,
 'num_epochs': 200,
 'batch_size': 32,
 'learning_rate': 0.00016847952401702993,
 'optimizer_scheduler': 'ReduceLROnPlateau'}

In [3]:
with open(r"gin_optuna_50_20250822.pkl", "rb") as f:
    all_folds_best_params = joblib.load(f)

In [4]:
all_folds_best_params

{'fold_1': {'best_params': {'num_layers': 1,
   'hidden_size': 256,
   'conv_dropout_rate': 0.3,
   'classifier_dropout_rate': 0.1,
   'use_layer_norm': False,
   'pool_hidden_size': 256,
   'num_epochs': 200,
   'batch_size': 32,
   'learning_rate': 0.00016847952401702993,
   'optimizer_scheduler': 'ReduceLROnPlateau'},
  'train_tracker': {'acc': tensor([0.7277, 0.7625, 0.7775, 0.7889, 0.7995, 0.8089, 0.8177, 0.8249, 0.8331,
           0.8412, 0.8464, 0.8532, 0.8572, 0.8634, 0.8664, 0.8718, 0.8751, 0.8792,
           0.8831, 0.8856, 0.8887, 0.8914, 0.8940, 0.8943, 0.8960, 0.8986, 0.9003,
           0.9017, 0.9036, 0.9049, 0.9054, 0.9081, 0.9088, 0.9097, 0.9112, 0.9113,
           0.9119, 0.9136, 0.9145, 0.9142, 0.9151, 0.9160, 0.9153, 0.9164, 0.9176,
           0.9182, 0.9187, 0.9186, 0.9188, 0.9196, 0.9200, 0.9198, 0.9200, 0.9206,
           0.9246, 0.9243, 0.9243, 0.9252, 0.9253, 0.9260, 0.9257, 0.9265, 0.9274,
           0.9262, 0.9259, 0.9259, 0.9288, 0.9298, 0.9284, 0.9293, 0.929

In [17]:
[(max(item.get("val_tracker").get("acc")).item(), max(item.get("val_tracker").get("f1")).item(), max(item.get("val_tracker").get("auroc")).item()) for item in all_folds_best_params.values()]

[(0.8739550113677979, 0.874706506729126, 0.9335613250732422),
 (0.8745980858802795, 0.875756025314331, 0.9335613250732422),
 (0.8774919509887695, 0.8781061172485352, 0.9345682859420776),
 (0.8774919509887695, 0.8781061172485352, 0.9345682859420776),
 (0.8774919509887695, 0.8781061172485352, 0.9362185597419739),
 (0.8774919509887695, 0.8781061172485352, 0.9362185597419739),
 (0.8774919509887695, 0.8781061172485352, 0.9362185597419739),
 (0.8774919509887695, 0.8781061172485352, 0.9362185597419739),
 (0.8774919509887695, 0.8781061172485352, 0.9362185597419739)]

In [30]:
all_folds_best_params["fold_9"]["val_tracker"]["auroc"][:20]

tensor([0.8290, 0.8374, 0.8529, 0.8659, 0.8759, 0.8838, 0.8891, 0.8917, 0.9010,
        0.9009, 0.9061, 0.9082, 0.9112, 0.9141, 0.9146, 0.9208, 0.9209, 0.9184,
        0.9210, 0.9214, 0.9221, 0.9251, 0.9274, 0.9264, 0.9285, 0.9250, 0.9272,
        0.9265, 0.9274, 0.9274, 0.9268, 0.9272, 0.9266, 0.9301, 0.9306, 0.9296,
        0.9282, 0.9291, 0.9290, 0.9303, 0.9305, 0.9290, 0.9315, 0.9299, 0.9298,
        0.9299, 0.9303, 0.9292, 0.9288, 0.9283, 0.9307, 0.9315, 0.9312, 0.9306,
        0.9334, 0.9332, 0.9320, 0.9314, 0.9329, 0.9310, 0.9328, 0.9310, 0.9315,
        0.9295, 0.9324, 0.9308, 0.9329, 0.9321, 0.9319, 0.9312, 0.9336, 0.9320,
        0.9312, 0.9329, 0.9326, 0.9321, 0.9330, 0.9316, 0.9323, 0.9315, 0.9324,
        0.9318, 0.9317, 0.9320, 0.9318, 0.9322, 0.9331, 0.9318, 0.9327, 0.9328,
        0.9320, 0.9328, 0.9319, 0.9330, 0.9324, 0.9327, 0.9334, 0.9325, 0.9322,
        0.9322, 0.9320, 0.9328, 0.9325, 0.9325, 0.9320, 0.9324, 0.9322, 0.9323,
        0.9317, 0.9329, 0.9324, 0.9320, 

In [33]:
from copy import deepcopy
all_folds_best_params_updated = deepcopy(all_folds_best_params)
for i in range(1,10):
    all_folds_best_params_updated[f"fold_{i}"]["val_tracker"]["auroc"] = all_folds_best_params[f"fold_{i}"]["val_tracker"]["auroc"][(i-1) * 200: i * 200]
    all_folds_best_params_updated[f"fold_{i}"]["val_tracker"]["f1"] = all_folds_best_params[f"fold_{i}"]["val_tracker"]["f1"][(i-1) * 200: i * 200]
    all_folds_best_params_updated[f"fold_{i}"]["val_tracker"]["acc"] = all_folds_best_params[f"fold_{i}"]["val_tracker"]["acc"][(i-1) * 200: i * 200]
    all_folds_best_params_updated[f"fold_{i}"]["train_tracker"]["auroc"] = all_folds_best_params[f"fold_{i}"]["train_tracker"]["auroc"][(i-1) * 200: i * 200]
    all_folds_best_params_updated[f"fold_{i}"]["train_tracker"]["f1"] = all_folds_best_params[f"fold_{i}"]["train_tracker"]["f1"][(i-1) * 200: i * 200]
    all_folds_best_params_updated[f"fold_{i}"]["train_tracker"]["auraccoc"] = all_folds_best_params[f"fold_{i}"]["train_tracker"]["acc"][(i-1) * 200: i * 200]

In [39]:
metrics = [(max(item.get("val_tracker").get("acc")).item(), max(item.get("val_tracker").get("f1")).item(), max(item.get("val_tracker").get("auroc")).item()) for item in all_folds_best_params_updated.values()]

In [42]:
import pandas as pd
pd.DataFrame(metrics, columns=["val_acc", "val_f1", "val_auroc"], index=[f"fold_{i}" for i in range(1,10)])

,val_acc,val_f1,val_auroc
fold_1,0.873955,0.874707,0.933561
fold_2,0.874598,0.875756,0.932621
fold_3,0.877492,0.878106,0.934568
fold_4,0.874277,0.875078,0.933621
fold_5,0.874920,0.875968,0.936219
fold_6,0.872026,0.872553,0.930099
fold_7,0.870900,0.871803,0.933130
fold_8,0.871684,0.872639,0.931324
fold_9,0.870076,0.871176,0.930017


In [38]:
all_folds_best_params_updated["fold_1"]

{'best_params': {'num_layers': 1,
  'hidden_size': 256,
  'conv_dropout_rate': 0.3,
  'classifier_dropout_rate': 0.1,
  'use_layer_norm': False,
  'pool_hidden_size': 256,
  'num_epochs': 200,
  'batch_size': 32,
  'learning_rate': 0.00016847952401702993,
  'optimizer_scheduler': 'ReduceLROnPlateau'},
 'train_tracker': {'acc': tensor([0.7277, 0.7625, 0.7775, 0.7889, 0.7995, 0.8089, 0.8177, 0.8249, 0.8331,
          0.8412, 0.8464, 0.8532, 0.8572, 0.8634, 0.8664, 0.8718, 0.8751, 0.8792,
          0.8831, 0.8856, 0.8887, 0.8914, 0.8940, 0.8943, 0.8960, 0.8986, 0.9003,
          0.9017, 0.9036, 0.9049, 0.9054, 0.9081, 0.9088, 0.9097, 0.9112, 0.9113,
          0.9119, 0.9136, 0.9145, 0.9142, 0.9151, 0.9160, 0.9153, 0.9164, 0.9176,
          0.9182, 0.9187, 0.9186, 0.9188, 0.9196, 0.9200, 0.9198, 0.9200, 0.9206,
          0.9246, 0.9243, 0.9243, 0.9252, 0.9253, 0.9260, 0.9257, 0.9265, 0.9274,
          0.9262, 0.9259, 0.9259, 0.9288, 0.9298, 0.9284, 0.9293, 0.9291, 0.9292,
          0.9308,

In [43]:
with open(r"all_folds_best_params_results_20250822.pkl", "wb") as f:
    joblib.dump(all_folds_best_params_updated, f)

In [1]:
import joblib
with open(r"optuna_results_20250829_113428.pkl", "rb") as f:
    optuna_results = joblib.load(f)

In [7]:
[item.values for item in optuna_results["fold_0"]["study"].trials]

[[0.942287802696228],
 [0.9297704696655273],
 [0.9421130418777466],
 [0.9351833462715149],
 [0.9437847137451172],
 [0.9308838844299316],
 [0.9209376573562622],
 [0.9391503930091858],
 [0.8790242671966553],
 [0.9406715631484985],
 [0.8805596828460693],
 [0.9384638667106628],
 [0.9430910348892212],
 [0.9247027039527893],
 [0.9392178058624268],
 [0.947102427482605],
 [0.9139909744262695],
 [0.950417160987854],
 [0.9405863285064697],
 [0.9472415447235107],
 [0.926308810710907],
 [0.9480395317077637],
 [0.9481739401817322],
 [0.9478071331977844],
 [0.9447917938232422],
 [0.8254144191741943],
 [0.9459085464477539],
 [0.9492442607879639],
 [0.9419183731079102],
 [0.9419786930084229]]

In [8]:
optuna_results["fold_0"]["study"].best_trial

FrozenTrial(number=17, state=1, values=[0.950417160987854], datetime_start=datetime.datetime(2025, 8, 29, 4, 5, 33, 957851), datetime_complete=datetime.datetime(2025, 8, 29, 4, 35, 46, 483199), params={'num_layers': 1, 'hidden_size': 460, 'conv_dropout_rate': 0.4995385821869766, 'classifier_dropout_rate': 0.347175534027804, 'use_layer_norm': False, 'pool_hidden_size': 286, 'num_epochs': 196, 'batch_size': 127, 'learning_rate': 0.0014567361305870153, 'optimizer_scheduler': 'OneCycleLR'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'num_layers': IntDistribution(high=3, log=False, low=1, step=1), 'hidden_size': IntDistribution(high=512, log=False, low=128, step=1), 'conv_dropout_rate': FloatDistribution(high=0.5, log=False, low=0.01, step=None), 'classifier_dropout_rate': FloatDistribution(high=0.5, log=False, low=0.01, step=None), 'use_layer_norm': CategoricalDistribution(choices=(True, False)), 'pool_hidden_size': IntDistribution(high=1024, log=False, low=128,

In [3]:
import optuna
study_name = "optuna_gine_fold_0"
storage = "sqlite:///gnn_optuna_database/optuna_gine_fold_0.db"

study = optuna.load_study(study_name=study_name, storage=storage)

In [4]:
study.best_params

{'num_layers': 1,
 'hidden_size': 460,
 'conv_dropout_rate': 0.4995385821869766,
 'classifier_dropout_rate': 0.347175534027804,
 'use_layer_norm': False,
 'pool_hidden_size': 286,
 'num_epochs': 196,
 'batch_size': 127,
 'learning_rate': 0.0014567361305870153,
 'optimizer_scheduler': 'OneCycleLR'}